In [42]:
import os
import sys
import mysql.connector
import configparser
import subprocess
import pandas as pd
from pathlib import Path
import csv
from datetime import datetime
import sqlite3
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

class CbsExtraction():
    
    def __init__(self, database_path , output_path):
        self.database_path = database_path
        self.output_path = output_path

    # get database connection parameters from config file
    def get_db_params(self):
        config = configparser.ConfigParser()
        config.read('config.ini') 
        user = config.get('Database', 'user')
        password = config.get('Database', 'password')
        host = config.get('Database', 'host')
        port = config.get('Database', 'port')
        return user, password, host, port
    
    # create connection to database
    def get_connection(self):
         user, password, host, port  = self.get_db_params()
         connection = mysql.connector.connect(user=user, password=password, host=host, port=port)
         return connection
    
     # remove database mrs from sql file 
    def trim_database(self, database_name):
        with open(database_name, "r", encoding='ISO-8859-1') as f:
            content = f.read()
        start_text = "USE `mrs`;"
        updated_text = content.split(start_text)[0]
        directory, filename = os.path.split(database_name)
        new_filename = f"modified_{filename}"
        file_path = os.path.join(directory, new_filename)
        with open(file_path, "w",encoding='utf-8') as f:
            f.write(updated_text)
        return file_path
    
    # Restore database
    def restore_database(self, database_name):
        database_name = database_name.replace("\\", "/")
        # Helper function for deleting existing schemas
        def drop_database(cursor, schema):
            schemas_to_drop = ['client', 'consultation', 'deduplication', 'facility', 'mrs', 'provider', 'report', 'terminology', 'zimepms']
            if schema in schemas_to_drop:
                cursor.execute(f'DROP DATABASE {schema}')
                print(f"   >>> DROPPED [{schema}]")
        # Establish connection to the server
        connection = self.get_connection()
        # Drop existing EHR schemas before restoring new database
        with connection.cursor() as cursor:
            cursor.execute('SET foreign_key_checks = 0')
            cursor.execute('SELECT SCHEMA_NAME FROM information_schema.schemata;')
            schemas = [row[0] for row in cursor.fetchall()]
            for schema in schemas:
                drop_database(cursor, schema)
        # Get DB credentials 
        user, password , host, port = self.get_db_params()
        restore_command = f"mysql -u {user} -p{password} -h {host} -P {port} < {database_name}"
        try:
            # print(" >>> Restoring database: "+restore_command)
            subprocess.run(restore_command, shell=True, check=True)
            print('   >>> DATABASE RESTORE [SUCCESSFUL>>>]')
        except Exception as e:
            log_file = os.path.join(os.getcwd(), 'logs.txt')
            with open(log_file, 'a') as f:
                f.write(f'{database_name} {str(e)}\n')
    
    def get_mapping_file(self):
        mapping_file = pd.read_csv("mapping_file.csv")
        mapping_file['Facility ID'] = mapping_file['Facility ID'].str.strip()
        return mapping_file
    
    def get_folder_size(self):
        total_size = 0
        for path, dirs, files in os.walk(self.database_path):
            for file in files:
                file_path = os.path.join(path, file)
                total_size += os.path.getsize(file_path)
        size_in_kb = total_size / 1024
        size_in_mb = size_in_kb / 1024
        size_in_gb = size_in_mb / 1024
        return round(size_in_gb,2)
    
    def get_database_size(self,filename):
        size = round(Path( filename).stat().st_size /(1024*1024*1024),2)
        return size
    
    def get_facility_databases(self): 
        facility_databases = []
        for root, dirs, files in os.walk(self.database_path):
            for file in files:
                if file.endswith(".sql") and not file.startswith("modified"):
                    file_path = os.path.join(root, file)
                    facility_databases.append(file_path)
        return facility_databases
    
    def extracting_data(self,sql,connection):
        try:
            connection = self.get_connection()
            df = pd.read_sql(sql,connection)
            connection.close()
            return df
        except:  # noqa: E722
            return None
        
    def get_facility_details(self,mapping_file,facility_id):
        facility_name = mapping_file.loc[mapping_file['Facility ID'] == facility_id] ["Facility"].values
        if facility_name.size > 0:
            facility_name = facility_name[0]
            district_name = mapping_file.loc[mapping_file['Facility ID'] == facility_id] ["District"].values
            if district_name.size > 0:
                district_name = district_name[0]
            else:
                district_name = ""
            province_name = mapping_file.loc[mapping_file['Facility ID'] == facility_id] ["Province"].values
            if province_name.size > 0:
                province_name = province_name[0]
            else:
                province_name = ""
        else:
            facility_name = ""
            district_name = ""
            province_name = ""
        return facility_name,district_name,province_name
    
    # def clean_string(self,text):
    #     if text == "empty string":
    #        return np.nan
    #     pattern = r'[^a-zA-Z0-9\s]'  # Matches any character that is not alphanumeric or whitespace
    #     cleaned_text = re.sub(pattern, '', text).lower().replace(' ','')
    #     return cleaned_text
    
    def match_on_lab(self,df_ehr,df_nmrl_filtered, df_nmrl):
        '''get only lab request numbers that are unique in EHR'''
        only_one_lab_id = [item for item in list(df_ehr["clean_laboratory_request_number"]) if list(df_ehr["clean_laboratory_request_number"]).count(item) == 1]
        '''matches in nmrl'''
        df_nmrl_match = df_nmrl_filtered[df_nmrl_filtered['clean_laboratory_request_number'].isin(only_one_lab_id)]
        matches_found = list(set(df_nmrl_match['clean_laboratory_request_number']))
        '''matches in EHR'''
        df_ehr_match = df_ehr[df_ehr['clean_laboratory_request_number'].isin(matches_found)]
        '''get dataframe without matched lab numbers'''
        df_nmrl_not_matched = df_nmrl[~df_nmrl['clean_laboratory_request_number'].isin(matches_found)]
        # create dataframe to save , dataframe contains matched records 
        df_matched = pd.merge(df_ehr_match ,df_nmrl_match, on = 'clean_laboratory_request_number', how = 'left')
        print(f"""    {df_ehr_match.shape[0]} matches found among {df_positives.shape[0]} records {df_ehr_match.shape[0]/df_positives.shape[0] * 100:.2f} % in EHR ART data based on lab request number""")
        print(f"""    {df_nmrl_match.shape[0]} matches found among {df_nmrl_filtered.shape[0]} records {df_nmrl_match.shape[0]/df_nmrl_filtered.shape[0] * 100:.2f} % in NMRL data based on lab request number""")
        df_matched = df_matched.rename(columns={
                                               'sample_date':'Date at which Viral Load result was issued',
                                                'Date of Dispatch': 'Date for which Viral Load was taken' ,
                                                'Result':'Viral Load result'})
        Viral_Load_Sample_submitted_to_lab = 'TRUE'
        Was_Viral_Load_result_issued = 'TRUE'
        df_matched['Viral Load Sample submitted to lab'] = Viral_Load_Sample_submitted_to_lab
        df_matched['Was Viral Load result issued'] = Was_Viral_Load_result_issued
        df_matched['Event_date'] = df_matched['Date at which Viral Load result was issued']
        df_matched = df_matched[[
                 'Event_date', 'person_id',
                'Date at which Viral Load result was issued',
                'Date for which Viral Load was taken',
                'Viral Load Sample submitted to lab', 'Was Viral Load result issued',
                'Viral Load result'
        ]]
        return  df_matched, df_nmrl_not_matched
    
    # This function finds matches using exact match in art number
    def match_on_art_number(self,df_ehr, df_nmrl):
        '''Get art numbers from the EHR where each art number has only one person ID'''
        only_one_person_id = df_ehr.groupby('clean_art_number').filter(lambda x: x['person_id'].nunique() == 1)['clean_art_number']      
        '''matches in nmrl'''
        df_nmrl_match = df_nmrl[df_nmrl['clean_art_number'].isin(only_one_person_id)]
        matches_found = list(set(df_nmrl_match['clean_art_number']))
        '''matches in EHR'''
        df_ehr_match = df_ehr[df_ehr['clean_art_number'].isin(matches_found)]
        '''get dataframe without matched art numbers'''
        df_nmrl_not_matched = df_nmrl[~df_nmrl['clean_art_number'].isin(matches_found)]
        # create dataframe to save , dataframe contains matched records 
        df_matched = pd.merge(df_nmrl_match, df_ehr_match, on = 'clean_art_number', how = 'left')
        print(f"""    {df_ehr_match.shape[0]} matches found among {df_positives.shape[0]} records {df_ehr_match.shape[0]/df_positives.shape[0] * 100:.2f} % in EHR ART data based on ART number""")
        print(f"""    {df_nmrl_match.shape[0]} matches found among {df_nmrl.shape[0]} records {df_nmrl_match.shape[0]/df_nmrl.shape[0] * 100:.2f} % in NMRL data based on ART number""")
        df_matched = df_matched.rename(columns={'Art Province Name': 'Viral Load Province Name', 
                                                'Art District Name': 'Viral Load District Name',
                                                'Art Facility Name': 'Viral Load Facility Name',
                                                'Art Facility ID': 'Viral Load Facility ID',
                                                'Art DB Name': 'Viral Load DB Name',
                                               'sample_date':'Date at which Viral Load result was issued',
                                                'Date of Dispatch': 'Date for which Viral Load was taken' ,
                                                'Result':'Viral Load result'})
        Viral_Load_Sample_submitted_to_lab = 'TRUE'
        Was_Viral_Load_result_issued = 'TRUE'
        df_matched['Viral Load Sample submitted to lab'] = Viral_Load_Sample_submitted_to_lab
        df_matched['Was Viral Load result issued'] = Was_Viral_Load_result_issued
        df_matched['Event_date'] = df_matched['Date at which Viral Load result was issued']
        df_matched = df_matched[[
                'Viral Load Province Name', 'Viral Load District Name',
                'Viral Load Facility Name', 'Viral Load Facility ID',
                'Viral Load DB Name', 'Event_date', 'person_id',
                'Date at which Viral Load result was issued',
                'Date for which Viral Load was taken',
                'Viral Load Sample submitted to lab', 'Was Viral Load result issued',
                'Viral Load result'
        ]]
        return  df_matched, df_nmrl_not_matched
    
    def match_on_demos(self,df_ehr, df_nmrl):
        # define the variables to use for matching
        match_variables = ['LastName','FirstName','Sex','Birthdate_cleaned']
        '''convert match variables to lower for comaprison'''
        for column in match_variables[:3]:
            df_ehr[column] = df_ehr[column].str.lower()
            df_nmrl[column] = df_nmrl[column].str.lower() 
        '''Get person_id from the EHR where each set of match variables has only one person ID'''
        df_only_one_person_id = df_ehr.groupby(match_variables).filter(lambda x: x['person_id'].nunique() == 1) [['person_id','LastName','FirstName','Sex','Birthdate_cleaned']]  
        '''get list of lists for df_only_one_person_id'''
        '''matches in nmrl'''
        df_nmrl_match = pd.merge(df_nmrl, df_only_one_person_id , on = match_variables, how = 'inner')
        sample_ids = df_nmrl_match['Sample ID']
        '''matches in EHR'''
        df_ehr_match_count = len(set(df_nmrl_match['person_id']))
        df_nmrl_not_matched = df_nmrl[~df_nmrl['Sample ID'].isin(sample_ids)]
        # create dataframe to save , dataframe contains matched records 
        df_nmrl_match = df_nmrl_match[['Sample ID', 'FirstName','LastName', 
                                       'Sex', 'Date of Dispatch', 'Result','sample_date','Birthdate_cleaned']]
        
        df_ehr = df_ehr[[
                'Province Name', 'District Name', 'Facility Name', 'Facility Id',
                'Facility DB Name', 'person_id', 'FirstName', 'LastName',
                'Sex',  'Birthdate_cleaned'
        ]]
        df_matched = pd.merge(df_nmrl_match, df_ehr, on = match_variables, how = 'left')
        df_nmrl_not_matched.drop('Birthdate_cleaned',axis = 1,inplace=True)
        print(f"""   {df_ehr_match_count} matches found among {df_positives.shape[0]} records {df_ehr_match_count/df_positives.shape[0] * 100:.2f} % in EHR data based on demographics """)
        print(f"""   {df_nmrl_match.shape[0]} matches found among {df_nmrl.shape[0]} records {df_nmrl_match.shape[0]/df_nmrl.shape[0] * 100:.2f} % in NMRL data based on ART number""")
        df_matched = df_matched.rename(columns={'Province Name': 'Viral Load Province Name', 
                                                'District Name': 'Viral Load District Name',
                                                'Facility Name': 'Viral Load Facility Name',
                                                'Facility Id': 'Viral Load Facility ID',
                                                'Facility DB Name': 'Viral Load DB Name',
                                               'sample_date':'Date at which Viral Load result was issued',
                                                'Date of Dispatch': 'Date for which Viral Load was taken' ,
                                                'Result':'Viral Load result'})
        Viral_Load_Sample_submitted_to_lab = 'TRUE'
        Was_Viral_Load_result_issued = 'TRUE'
        df_matched['Viral Load Sample submitted to lab'] = Viral_Load_Sample_submitted_to_lab
        df_matched['Was Viral Load result issued'] = Was_Viral_Load_result_issued
        df_matched['Event_date'] = df_matched['Date at which Viral Load result was issued']
        df_matched = df_matched[[
                'Viral Load Province Name', 'Viral Load District Name',
                'Viral Load Facility Name', 'Viral Load Facility ID',
                'Viral Load DB Name', 'Event_date', 'person_id',
                'Date at which Viral Load result was issued',
                'Date for which Viral Load was taken',
                'Viral Load Sample submitted to lab', 'Was Viral Load result issued',
                'Viral Load result'
        ]]
        return  df_matched, df_nmrl_not_matched
    
    ''' Functions for cleaning up dates '''
    ''' Function for extracting the year from DOB and fixing typos/unrealistic values '''
    def fix_years(self,df_in, date_column='DOB'):
        # Fix any years that are not 4 digits long
        incorrect_length = (df_in['YYYY'].apply(lambda x: len(str(x)) != 4))
        for i in incorrect_length[incorrect_length].index:
            x = df_in.loc[i, 'YYYY']
            x = str(x)
            if len(x) > 4:
                y = x[:4]
            elif len(x) in [2,3]:
                if x[:2] == '19':
                    y = '1980'
                elif x[:2] == '20':
                    y = '2000'
                else:
                    if len(x) == 3:
                        y = '{}0'.format(x)
                    else:
                        y = '00{}'.format(x)
            else:
                y = '1980'
            df_in.loc[i, 'YYYY'] = y
        # Fix any years that don't start with 1 or 2
        incorrect_first_digit = (df_in['YYYY'].apply(lambda x: str(x)[0] not in ['1','2']))
        for i in incorrect_first_digit[incorrect_first_digit].index:
            x = df_in.loc[i,'YYYY']
            x = str(x)
            if not x.isnumeric():
                y = '1980'
            else:
                y = x[1:]+x[0]
            df_in.loc[i,'YYYY'] = y        

        df_in['YYXX'] = df_in['YYYY'].apply(lambda x: x[:2]) # extract first two digits of year
        df_in['XXYY'] = df_in['YYYY'].apply(lambda x: x[-2:]) # extract last two digits of year
        df_in['new_YYXX'] = df_in['YYXX']
        df_in.loc[(~df_in['YYXX'].isin(['19','20'])) &
                    (df_in['XXYY'].astype(int) > int(str(pd.to_datetime('today').year)[-2:])), 'new_YYXX'] = '19'
        df_in.loc[(~df_in['YYXX'].isin(['19','20'])) &
                    (df_in['XXYY'].astype(int) <= int(str(pd.to_datetime('today').year)[-2:])), 'new_YYXX'] = '20'
        df_in.loc[(df_in['YYXX'] == '20') &
                    (df_in['XXYY'].astype(int) > int(str(pd.to_datetime('today').year)[-2:])), 'new_YYXX'] = '19'
        df_in['new_YYYY'] = df_in['new_YYXX'] + df_in['XXYY']
        return df_in


    def clean_dates(self,df_in, date_column, plot_years=True):
        # Wrapper function for fix_years, plus some other cleaning
        ''' Dates formatted with a slash ('/') are in the format DD/MM/YYYY or MM/DD/YYYY '''
        ''' Dates formatted with a dash ('-') are in the format YYYY-MM-DD '''
        ''' We need to handle these different formats separately '''
        original_index = df_in.index
        # df_in = df_in.reset_index(drop=True)
    
        dob_slash = df_in[df_in[date_column].astype(str).str.contains('/')][[date_column]]
        dob_dash = df_in[df_in[date_column].astype(str).str.contains('-')][[date_column]]
        # there are some patients without a documented DOB
        dob_none = df_in[(~df_in[date_column].astype(str).str.contains('/')) &
                        (~df_in[date_column].astype(str).str.contains('-'))][[date_column]]

        ''' Clean up dates formatted like DD/MM/YYYY or MM/DD/YYYY '''
        dob_slash['YYYY'] = dob_slash[date_column].apply(lambda x: str(x).split('/')[-1]) # extract year
        dob_slash = self.fix_years(dob_slash, date_column)
        dob_slash['new_{}'.format(date_column)] = pd.to_datetime(
            dob_slash[date_column].apply(lambda x: '/'.join(x.split('/')[:2])) + '/' + dob_slash['new_YYYY'],
            format= 'mixed',
            dayfirst=True)

        ''' Clean up dates formatted like YYYY-MM-DD '''
        dob_dash['YYYY'] = dob_dash[date_column].apply(lambda x: str(x).split('-')[0]) # extract year
        dob_dash = self.fix_years(dob_dash, date_column)
        dob_dash['new_YYYY'] = dob_dash['new_YYXX'] + dob_dash['XXYY']
        dob_dash['new_{}'.format(date_column)] = pd.to_datetime(
            dob_dash['new_YYYY'] + '-' + dob_dash[date_column].apply(lambda x: '-'.join(x.split('-')[-2:])),
            dayfirst=False)
        if plot_years:
            pass
            # fig, ax = plt.subplots()
            # dob_slash['new_YYYY'].astype(int).value_counts().sort_index().plot(style='.', ax=ax, label='slash')
            # dob_dash['new_YYYY'].astype(int).value_counts().sort_index().plot(style='.', ax=ax, label='dash')
            # plt.title(date_column)
            # plt.legend()
        return pd.concat([dob_dash, dob_slash, dob_none], axis=0)['new_{}'.format(date_column)].loc[original_index]

    # def fix_years(self, df_in, date_column='DOB'):
    #     # Fix any years that are not 4 digits long
    #     incorrect_length = (df_in['YYYY'].apply(lambda x: len(str(x)) != 4))
    #     for i in incorrect_length[incorrect_length].index:
    #         x = df_in.loc[i, 'YYYY']
    #         x = str(x)
    #         if len(x) > 4:
    #             y = x[:4]
    #         elif len(x) in [2,3]:
    #             if x[:2] == '19':
    #                 y = '1980'
    #             elif x[:2] == '20':
    #                 y = '2000'
    #             else:
    #                 if len(x) == 3:
    #                     y = '{}0'.format(x)
    #                 else:
    #                     y = '00{}'.format(x)
    #         else:
    #             y = '1980'
    #         df_in.loc[i, 'YYYY'] = y
    #     # Fix any years that don't start with 1 or 2
    #     incorrect_first_digit = (df_in['YYYY'].apply(lambda x: str(x)[0] not in ['1','2']))
    #     for i in incorrect_first_digit[incorrect_first_digit].index:
    #         x = df_in.loc[i,'YYYY']
    #         x = str(x)
    #         if not x.isnumeric():
        #         y = '1980'
        #     else:
        #         y = x[1:]+x[0]
        #     df_in.loc[i,'YYYY'] = y        
        # df_in['YYXX'] = df_in['YYYY'].apply(lambda x: x[:2]) # extract first two digits of year
        # df_in['XXYY'] = df_in['YYYY'].apply(lambda x: x[-2:]) # extract last two digits of year
        # df_in['new_YYXX'] = df_in['YYXX']
        # df_in.loc[(~df_in['YYXX'].isin(['19','20'])) &
        #             (df_in['XXYY'].astype(int) > int(str(pd.to_datetime('today').year)[-2:])), 'new_YYXX'] = '19'
        # df_in.loc[(~df_in['YYXX'].isin(['19','20'])) &
        #             (df_in['XXYY'].astype(int) <= int(str(pd.to_datetime('today').year)[-2:])), 'new_YYXX'] = '20'
        # df_in.loc[(df_in['YYXX'] == '20') &
        #             (df_in['XXYY'].astype(int) > int(str(pd.to_datetime('today').year)[-2:])), 'new_YYXX'] = '19'
        # df_in['new_YYYY'] = df_in['new_YYXX'].astype(str)  + df_in['XXYY'].astype(str) 
        # return df_in
    

    # def clean_dates(self,df_in, date_column = 'DOB', plot_years=True):
    #     # Wrapper function for fix_years, plus some other cleaning
    #     ''' Dates formatted with a slash ('/') are in the format DD/MM/YYYY or MM/DD/YYYY '''
    #     ''' Dates formatted with a dash ('-') are in the format YYYY-MM-DD '''
    #     ''' We need to handle these different formats separately '''  
    #     dob_slash = df_in[df_in[date_column].astype(str).str.contains('/')][[date_column]]
    #     dob_dash = df_in[df_in[date_column].astype(str).str.contains('-')][[date_column]]
    #     # there are some patients without a documented DOB
    #     dob_none = df_in[(~df_in[date_column].astype(str).str.contains('/')) &
    #                     (~df_in[date_column].astype(str).str.contains('-'))][[date_column]]
    #     ''' Clean up dates formatted like DD/MM/YYYY or MM/DD/YYYY '''
    #     dob_slash['YYYY'] = dob_slash[date_column].apply(lambda x: str(x).split('/')[-1]) # extract year
    #     dob_slash = self.fix_years(dob_slash, date_column)
    #     if (not dob_slash.empty):
        #     dob_slash['new_{}'.format(date_column)] = pd.to_datetime(
        #         dob_slash[date_column].apply(lambda x: '/'.join(str(x).split('/')[:2])) + '/' + dob_slash['new_YYYY'].astype(str), 
        #              format='mixed', 
        #         infer_datetime_format=True,
        #         dayfirst=True)
        # ''' Clean up dates formatted like YYYY-MM-DD '''
        # dob_dash['YYYY'] = dob_dash[date_column].apply(lambda x: str(x).split('-')[0]) # extract year
        # dob_dash = self.fix_years(dob_dash, date_column)
        # dob_dash['new_YYYY'] = dob_dash['new_YYXX'] + dob_dash['XXYY']
        # dob_dash['new_{}'.format(date_column)] = pd.to_datetime(
        #     dob_dash['new_YYYY'] + '-' + dob_dash[date_column].apply(lambda x: '-'.join(str(x).split('-')[-2:])),
        #     dayfirst=False)
        # return pd.concat([dob_dash, dob_slash, dob_none], axis=0).sort_index()['new_{}'.format(date_column)]
    

    def load_and_clean_nmrl(self,file_path, plot_years=True):
        def distinguish_duplicated_sample_ids(row, ind):
            return  str(row['Sample ID']) + "_" + str(ind)

        cols_to_use = ['Sample ID','client sample ID','FirstName','Middle_Surname','Third_Surname',
                       'Client Patient ID',	'DOB','Sex','Date Sampled',	'Date of Dispatch',	'Result','Facility Id']
        vl_full = pd.read_csv(file_path, low_memory=False, usecols= cols_to_use)
        # Drop duplicated rows on Sample Id and Client Patient id:
        vl_full = vl_full.drop_duplicates(subset=['Sample ID','Client Patient ID'], keep= 'first')
        duplicated_values = list(set(vl_full[vl_full['Sample ID'].duplicated(keep=False)]['Sample ID']))
        # Rename duplicated Sample ID's:
        vl_full_with_duplicates = vl_full[vl_full['Sample ID'].isin(duplicated_values)]
        vl_full_without_duplicates = vl_full[~vl_full['Sample ID'].isin(duplicated_values)]
        vl_full_with_duplicates['Sample ID'] = vl_full_with_duplicates.apply(lambda row: distinguish_duplicated_sample_ids(row, row.name), axis=1)
        vl_full = pd.concat([vl_full_with_duplicates,vl_full_without_duplicates])
        vl_full = vl_full.fillna('empty string')
        vl_full = vl_full.replace('', 'empty string')
        # print('NMRL data has {:,.0f} records'.format(vl_full.shape[0]))
        # Remove any non-alphanumeric characters from names:
        vl_full['FirstName'] = vl_full['FirstName'].astype(str).str.replace('\W', '', regex=True).apply(lambda x: x.lower().replace(' ',''))
        vl_full['Middle_Surname'] = vl_full['Middle_Surname'].astype(str).str.replace('\W', '', regex=True).apply(lambda x: x.lower().replace(' ',''))
        vl_full['Third_Surname'] = vl_full['Third_Surname'].astype(str).str.replace('\W', '', regex=True).apply(lambda x: x.lower().replace(' ',''))
        # Clean up Client Patient ID (ART #)
        vl_full['Client Patient ID'] = vl_full['Client Patient ID'].astype(str).str.replace('\W', '', regex=True).apply(lambda x: x.lower().replace(' ',''))
        vl_full.loc[:,'clean_art_number'] = vl_full['Client Patient ID']
        # If patient only has numeric data in their name, drop them for now
        vl_full.loc[vl_full['FirstName'].astype(str).str.isnumeric(), 'FirstName'] = np.nan
        vl_full.loc[vl_full['Middle_Surname'].astype(str).str.isnumeric(), 'Middle_Surname'] = np.nan
        vl_full.loc[vl_full['Third_Surname'].astype(str).str.isnumeric(), 'Third_Surname'] = np.nan
        for n in ['FirstName','Middle_Surname','Third_Surname']:
            vl_full.loc[vl_full[n].isna(), n] = np.nan
            vl_full.loc[vl_full[n]=='nan', n] = np.nan
        ''' Add cleaned dates to dataframe '''
        vl_full['Birthdate'] = self.clean_dates(vl_full, 'DOB')
        vl_full['sample_date'] = self.clean_dates(vl_full, 'Date Sampled')
        vl_full = vl_full.drop(columns=['DOB','Date Sampled'])
        ''' Do final cleaning and return '''
        vl_full = vl_full.replace(
            {'FirstName':{'MISSING':np.nan, '':np.nan},
            'Middle_Surname':{'MISSING':np.nan, '':np.nan},
            'Third_Surname':{'MISSING':np.nan, '':np.nan}}
        ).rename(
            columns={
                'Middle_Surname':'MiddleName',
                'Third_Surname':'LastName',
                'Client Patient ID':'Client_Patient_ID'}
        )
        return vl_full

    def log(self,processing_time,province="",district="",facility="",site_code="",filename="",first_date="",last_date="",
            db_size="",version="",comment="",ehr_type = "",people_registered = "",patients_attended = ""): 
         df = pd.DataFrame({"Time File Processed":[processing_time],
                            "province": [province],
                            "district": [district],
                            "facility": [facility],
                            "Site Code":[site_code],
                            "File Name":[filename],
                            "Date site first used EHR": [first_date],
                            "Date site last used EHR": [last_date],
                            "Database Size": [db_size],
                            "Version":[version],
                            "Comments":[comment],
                            "EHR Type":[ehr_type],
                            "Number of peole registered": [people_registered],
                            "Number of patients": [patients_attended]
                            })
         df.to_csv('log.csv', mode='a', index = False, header=None)
    
    
if __name__ == '__main__':
    cbs_extraction = CbsExtraction('./dbs', './')
    # 1. Reading mapping file
    try:
         mapping_file = cbs_extraction.get_mapping_file()
    except:  # noqa: E722
        print("Mapping file not found")
        pass



    
    # 2. Load NMRL file
    try:
         if os.path.exists("nmrl_reduced.csv"):
              df_nmrl = cbs_extraction.load_and_clean_nmrl('nmrl_reduced.csv')
         else:
              df_nmrl = cbs_extraction.load_and_clean_nmrl('nmrl_with_facility_id.csv')
    except:  # noqa: E722
        print("NMRL file not found")
      

    # 3 create log file
    if not Path(cbs_extraction.output_path + "log.csv").exists():
            log_file_header = [
                "Time File Processed",
                "Province",
                "District",
                "Facility",
                "Site Code",
                "File Name",
                "Date site first used EHR",
                "Date site last used EHR",
                "Database Size",
                "Version",
                "Comments",
                "EHR Type",
                "Number of peole registered in EHR",
                "Number of people without date of hiv test"
            ]
            with open(cbs_extraction.output_path + "log.csv",'w') as file:
                writer = csv.writer(file)
                writer.writerow(log_file_header)
    if Path(cbs_extraction.output_path + "log.csv").exists():
        df_facilities_= pd.read_csv("log.csv")
        df_facilities_done = list(df_facilities_["File Name"])
        df_site_codes_done = list(df_facilities_["Site Code"])

    # 4 a. get list of databases to be processed
    facilities = cbs_extraction.get_facility_databases()
    print(">>> Number of databases ", len(facilities))
    # 4 b. get total size of all the databases to be processed
    size_of_dbs = cbs_extraction.get_folder_size()
    print(">>> Total size:" , round(size_of_dbs,2) , " GB")

    # 5 create global dataframes
    if os.path.exists("global_df_vl_nmrl_art.csv"):
        global_df_stats = pd.read_csv("Stats.csv")
        global_df_vl_nmrl_lab = pd.read_csv("global_vl_nmrl_lab.csv")
        global_df_vl_nmrl_art = pd.read_csv("global_df_vl_nmrl_art.csv")
        global_df_vl_nmrl_demos = pd.read_csv("global_df_vl_nmrl_demos.csv")
        global_df_demographics = pd.read_csv("Demographics.csv")
        global_df_positives = pd.read_csv("global_df_positives.csv")
        global_df_hts_negative = pd.read_csv("global_df_negative.csv")
        global_df_cbs = pd.read_csv("global_df_cbs.csv")
        global_df_recency = pd.read_csv("global_df_recency.csv")
        global_df_art = pd.read_csv("global_df_art.csv")
        global_df_art_visit= pd.read_csv("global_df_art_visit.csv")
        global_df_art_current_status = pd.read_csv("global_df_art_current_status.csv")
        global_df_viral_load = pd.read_csv("global_df_viral_load.csv")
        global_df_cd4 = pd.read_csv("global_df_cd4.csv")
        global_df_tb  = pd.read_csv("global_df_tb.csv")
        global_df_tb_screening = pd.read_csv("global_df_tb_screening.csv")
        global_df_transfer_out = pd.read_csv("global_df_transfer_out.csv")
        global_df_art_who_stage = pd.read_csv("global_df_art_who_stage.csv")
        global_df_laboratory_request = pd.read_csv("global_df_laboratory_request.csv")
        global_mother_to_baby = pd.read_csv("global_mother_to_baby.csv")
        global_baby_to_mother = pd.read_csv("global_baby_to_mother.csv")
        global_df_art_current_status2 = pd.read_csv("global_df_art_current_status2.csv")
    else:
        global_df_stats = pd.DataFrame()
        global_df_demographics = pd.DataFrame()
        global_df_positives = pd.DataFrame()
        global_df_hts_negative = pd.DataFrame()
        global_df_cbs = pd.DataFrame()
        global_df_recency = pd.DataFrame()
        global_df_art = pd.DataFrame()
        global_df_art_visit = pd.DataFrame()
        global_df_art_current_status = pd.DataFrame()
        global_df_viral_load = pd.DataFrame()
        global_df_cd4 = pd.DataFrame()
        global_df_tb = pd.DataFrame()
        global_df_tb_screening = pd.DataFrame()
        global_df_transfer_out =  pd.DataFrame()
        global_df_art_who_stage = pd.DataFrame()
        global_df_laboratory_request = pd.DataFrame()
        global_df_vl_nmrl_art = pd.DataFrame()
        global_df_vl_nmrl_demos = pd.DataFrame()
        global_df_vl_nmrl_lab = pd.DataFrame()
        global_baby_to_mother = pd.DataFrame()
        global_mother_to_baby = pd.DataFrame()
        global_df_art_current_status2 = pd.DataFrame()

    db_size_left = size_of_dbs

    # 6 create connection to db 
    connection = cbs_extraction.get_connection()
    
    for count , facility  in enumerate(facilities):

        try:
            '''Logging'''
            count = count + 1
            print(".........................................................................................................")
            print(f">>> {count} / {len(facilities)} {facility}")
            nmrl_size = "{:,}".format(df_nmrl.shape[0])
            print(">>> Starting NMRL size", nmrl_size) 
            processing_time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
            db_size = cbs_extraction.get_database_size(facility)
            db_size_left = db_size_left - db_size
            print(f"db size {db_size} GB")

            # 7 . Skip facilities already processed
            if facility in df_facilities_done:
                print("  >>> facility already done")
                db_size_left = db_size_left - db_size
                # cbs_extraction.log(processing_time,"","","","",facility,"","",db_size,"","Already Processed","web","","")
                continue

            if db_size <= 0:
                print("  >>> Corrupt database")
                db_size_left = db_size_left - db_size
                cbs_extraction.log(processing_time,"","","","",facility,"","",db_size,"","Corrupt Database","web","","")
                continue

            # 8. Trim database
            print("  >>> trimming database")
            trimmed_db = cbs_extraction.trim_database(facility)

            # 9. Restore database
            print("  >>> restoring database")
            cbs_extraction.restore_database(trimmed_db)

            conn = sqlite3.connect(':memory:')

            # 10. Get Facility id and check if database is not empty
            df_facility = cbs_extraction.extracting_data("""
                                SELECT facility_id, time FROM consultation.patient
                                where time <= now()
                                order by time desc
                                limit 1               
                            """, connection)
            if df_facility.empty:
                print(">>> database is empty")
                cbs_extraction.log(processing_time,"","","","",facility,"","",db_size,"","Database is empty","web","","")
                continue
            facility_id, latest_timestamp = df_facility.values.tolist()[0]

            # 11. Get facility details (name , province name and district name)
            facility_name , district_name, province_name =  cbs_extraction.get_facility_details(mapping_file,facility_id)

             # 12. Extract data from all needed tables and populate data into dataframes
            print("   >>> Extract data from all needed tables")
            ''' a . hts'''
            hts_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.hts
                                                                """, connection)
            hts_df.to_sql('hts', conn, index=False)

            '''b. patient'''
            patient_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.patient
                                                                """, connection)
            patient_df.to_sql('patient', conn, index=False)

            '''c. hts screening'''
            hts_screening_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.hts_screening
                                                                """, connection)
            hts_screening_df.to_sql('hts_screening', conn, index=False)

            '''d. person investigation'''
            person_investigation_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.person_investigation
                                                                """, connection)
            person_investigation_df.to_sql('person_investigation', conn, index=False)

            '''e. cbs'''
            cbs_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.cbs
                                                                """, connection)
            cbs_df.to_sql('cbs', conn, index=False)

            '''f. sexual history'''
            sexual_history_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.sexual_history
                                                                """, connection)
            sexual_history_df.to_sql('sexual_history', conn, index=False)

            '''g. sexual history question'''
            sexual_history_question_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.sexual_history_question
                                                                """, connection)
            sexual_history_question_df.to_sql('sexual_history_question', conn, index=False)

            '''h. patient tb screening'''
            patient_tb_screening_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.patient_tb_screening
                                                                """, connection)
            patient_tb_screening_df.to_sql('patient_tb_screening', conn, index=False)

            '''i. patient client profile'''
            patient_client_profile_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.patient_client_profile
                                                                """, connection)
            if patient_client_profile_df is not None:
                patient_client_profile_df.to_sql('patient_client_profile', conn, index=False)

            '''j. art transfer out'''
            art_transfer_out_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.art_transfer_out
                                                                """, connection)
            art_transfer_out_df.to_sql('art_transfer_out', conn, index=False)

            '''k. tb'''
            tb_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.tb
                                                                """, connection)
            tb_df.to_sql('tb', conn, index=False)

            '''l. laboratory investigation test'''
            laboratory_investigation_test_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.laboratory_investigation_test
                                                                """, connection)
            laboratory_investigation_test_df.to_sql('laboratory_investigation_test', conn, index=False)

            '''m. art visit'''
            art_visit_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.art_visit
                                                                """, connection)
            art_visit_df.to_sql('art_visit', conn, index=False)

            '''n. art visit'''
            art_who_stage_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.art_who_stage
                                                                """, connection)
            art_who_stage_df.to_sql('art_who_stage', conn, index=False)

            '''o. art current status'''
            art_current_status_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.art_current_status
                                                                """, connection)
            art_current_status_df.to_sql('art_current_status', conn, index=False)

            '''p. art appointment'''
            art_appointment_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.art_appointment
                                                                """, connection)
            art_appointment_df.to_sql('art_appointment', conn, index=False)

            '''q. art'''
            art_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.art
                                                                """, connection)
            art_df.to_sql('art', conn, index=False)

            '''r. person diagnosis'''
            person_diagnosis_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.person_diagnosis
                                                                """, connection)
            person_diagnosis_df.to_sql('person_diagnosis', conn, index=False)

            '''s. laboratory_request_order'''
            laboratory_request_order_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.laboratory_request_order
                                                                """, connection)
            laboratory_request_order_df.to_sql('laboratory_request_order', conn, index=False)

            '''t. person'''
            demographics_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM client.person
                                                                """, connection)
            demographics_df.to_sql('demographics', conn, index=False)

            '''u. identification'''
            identification_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM client.identification
                                                                """, connection)
            identification_df.to_sql('identification', conn, index=False)

            '''v. phone'''
            phone_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM client.phone
                                                                """, connection)
            phone_df.to_sql('phone', conn, index=False)

            '''w. tb_screening'''
            tb_screening_df = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.patient_tb_screening
                                                                """, connection)
            
            tb_screening_df.to_sql('tb_screening', conn, index=False)


            '''x. art current status2'''
            art_current_status_df2 = cbs_extraction.extracting_data("""
                                                        SELECT * FROM consultation.art_current_status
                                                                """, connection)
            art_current_status_df2.to_sql('art_current_status', conn, index=False)









            # 13. Extract cbs data from conctructed data frames
            print("   >>> extracting cbs data from dataframes")
            '''a. extract positives from hts'''
            positives_hts_query = """
                            WITH ranked_messages AS (
                                SELECT
                                    h.time AS Event_date,
                                    h.patient_id,
                                    h.laboratory_investigation_id,
                                    p.test,
                                    p.person_id,
                                    h.hts_number AS 'HTS Number',
                                    h.reason_for_not_initiating_art,
                                    h.time AS 'Date of HIV Test',
                                    h.entry_point AS 'Entry Point',
                                    h.purpose AS 'Reason for HIV Test',
                                    h.reason_for_not_performing_test AS 'Reason for not doing recency test',
                                    h.entry_point_id AS 'Entry Point Id',
                                    h.hts_type AS 'Test Method',
                                    h.client_already_positive AS 'Already positive',
                                    h.client_already_on_art AS 'Already on art',
                                    p.result AS HIV_Result,
                                    h.pregnant AS 'Pregnant during HIV Test',
                                    RANK() OVER (PARTITION BY p.person_id ORDER BY h.time ASC) AS rnt,
                                    'hts' AS 'Source of Positive'
                                FROM hts h
                                INNER JOIN person_investigation p ON h.laboratory_investigation_id = p.person_investigation_id
                                WHERE p.result = 'POSITIVE' AND h.purpose <> 'Retesting for ART initiation'
                            )
                            SELECT *
                            FROM ranked_messages
                            WHERE rnt = 1;
                                """
            df_positives_from_hts = pd.read_sql_query(positives_hts_query, conn)
            df_positives_from_hts = df_positives_from_hts.drop_duplicates(subset=['person_id'])
            total_positives_in_hts = df_positives_from_hts.shape[0]
            no_date_of_hiv_test_count_hts = df_positives_from_hts[~df_positives_from_hts['Date of HIV Test'].notna()].shape[0]
            hts_no_date = df_positives_from_hts[~df_positives_from_hts['Date of HIV Test'].notna()]
            if no_date_of_hiv_test_count_hts > 0:
                print(f"   >>> {no_date_of_hiv_test_count_hts}/{total_positives_in_hts} records have no date of hiv test in hts")
            df_positives_from_hts = df_positives_from_hts[df_positives_from_hts['Date of HIV Test'].notna()]
            replace_dict = {1:"YES",0:"NO"}
            df_positives_from_hts.replace({
                                        "Already positive": replace_dict,
                                        "Already on art": replace_dict,
                                        "Pregnant during HIV Test": replace_dict
                                    }, inplace=True)
            df_positives_from_hts.to_sql('hts_positives', conn, index=False)
            
            '''b. extract positives from cbs'''
            positives_cbs_query = """
                                SELECT
                                    c.date_of_hiv_test AS 'Event_date',
                                    c.person_id,
                                    c.date_of_hiv_test AS 'Date of HIV Test',
                                    'POSITIVE' AS HIV_Result,
                                    'cbs' AS 'Source of Positive'
                                FROM cbs c
                                LEFT JOIN hts_positives hp ON c.person_id = hp.person_id
                                WHERE hp.person_id IS NULL; 
                            """
            df_positives_from_cbs = pd.read_sql_query(positives_cbs_query, conn)
            df_positives_from_cbs = df_positives_from_cbs.drop_duplicates(subset=['person_id'])
            total_positives_in_cbs = df_positives_from_cbs.shape[0]
            no_date_of_hiv_test_count_cbs = df_positives_from_cbs[~df_positives_from_cbs['Date of HIV Test'].notna()].shape[0]
            cbs_no_date = df_positives_from_cbs[~df_positives_from_cbs['Date of HIV Test'].notna()]
            if no_date_of_hiv_test_count_cbs > 0:
                print(f"   >>> {no_date_of_hiv_test_count_cbs}/{total_positives_in_cbs} records have no date of hiv test in cbs")
            df_positives_from_cbs = df_positives_from_cbs[df_positives_from_cbs['Date of HIV Test'].notna()]
            df_positives_from_cbs.to_sql('cbs_positives', conn, index=False)

            '''c. extract positives from art'''
            positives_art_query = """
                                SELECT
                                    a.date_of_hiv_test AS 'Event_date',
                                    a.person_id,
                                    a.date_of_hiv_test AS 'Date of HIV Test',
                                    'POSITIVE' AS HIV_Result,
                                    'ART' AS 'Source of Positive'
                                FROM art a
                                LEFT JOIN hts_positives hp ON a.person_id = hp.person_id
                                LEFT JOIN cbs_positives cp ON a.person_id = cp.person_id
                                WHERE hp.person_id IS NULL AND cp.person_id IS NULL;
                            """
            df_positives_from_art = pd.read_sql_query(positives_art_query, conn)
            df_positives_from_art = df_positives_from_art.drop_duplicates(subset=['person_id'])
            total_positives_in_art = df_positives_from_art.shape[0]
            no_date_of_hiv_test_count_art = df_positives_from_art[~df_positives_from_art['Date of HIV Test'].notna()].shape[0]
            art_no_date = df_positives_from_art[~df_positives_from_art['Date of HIV Test'].notna()]
            if no_date_of_hiv_test_count_art > 0:
                print(f"   >>> {no_date_of_hiv_test_count_art}/{total_positives_in_art} records have no date of hiv test in art")
            df_positives_from_art = df_positives_from_art[df_positives_from_art['Date of HIV Test'].notna()]
            
            '''Logging . check if there are no positives in the database'''
            if df_positives_from_hts.empty and df_positives_from_cbs.empty and df_positives_from_art.empty:
                print(">>> There are no positive cases in the database")
                cbs_extraction.log(processing_time,province_name,district_name,facility_name, facility_id, facility,"","",db_size,"","Failed,No positives ","web","","")
                continue
            
            '''d. Merge all positives from hts , cbs and art'''
            df_positives = pd.concat([df_positives_from_hts,df_positives_from_cbs,df_positives_from_art])

            '''e. extract hiv test results for first, second and third tests'''
            hts_investigations_query = """
                                WITH ranked_messages AS (
                                    SELECT
                                        h.laboratory_investigation_id,
                                        h.result,
                                        h.time,
                                        p.person_id,
                                        ROW_NUMBER() OVER (PARTITION BY  h.laboratory_investigation_id ORDER BY h.time ASC) AS rnt
                                    FROM person_investigation p
                                    LEFT JOIN laboratory_investigation_test h ON p.person_investigation_id = h.laboratory_investigation_id
                                )
                                SELECT *
                                FROM ranked_messages
                                WHERE laboratory_investigation_id IN (
                                    SELECT laboratory_investigation_id
                                    FROM hts_positives
                                );
                            """
            df_hts_investigations = pd.read_sql_query(hts_investigations_query, conn)    
            df_hts_investigations = df_hts_investigations[df_hts_investigations['rnt']<=3]
            df_hts_investigations['rnt'] = df_hts_investigations['rnt'].map(
                {1:"HIVtestOneResult", 2: "HIVtestTwoResult",3:"HIVtestThreeResult"})
            df_hts_investigations.drop_duplicates(subset=['person_id', 'rnt'], inplace=True)
            df_hts_investigations = df_hts_investigations.pivot(index=['person_id','laboratory_investigation_id'], columns=['rnt'], values='result')
            df_hts_investigations = df_hts_investigations.reset_index()
            column_order = {'person_id': 1, 
                                'HIVtestOneResult':2,'HIVtestTwoResult':3,
                                'HIVtestThreeResult':4,
                        'laboratory_investigation_id':5
                                }
            df_hts_investigations = df_hts_investigations[[col for col in sorted(df_hts_investigations.columns, key=lambda x: column_order.get(x, float('inf')))]]

            '''f. merge dataframe of positives with the investigations done'''
            df_positives_with_investigations = pd.merge(df_positives,df_hts_investigations, on =["laboratory_investigation_id","person_id"], how ="left")

            '''g. extract sexual status'''
            sexual_history_query = """
                                SELECT
                                    sh.sexual_history_id, sh.person_id, sh.patient_id, sh.sexually_active, sh.date
                                FROM sexual_history sh
                                LEFT JOIN hts_positives hp ON sh.patient_id = hp.patient_id
                                WHERE hp.patient_id is not NULL;
                            """
            df_sexual_history = pd.read_sql_query(sexual_history_query, conn)  
            replace_dict = {1:"YES",0:"NO"}
            df_sexual_history.replace({"sexually_active": replace_dict},inplace=True)
            df_sexual_history.to_sql('sexual_history_table', conn, index=False)
            
            '''h. extract sexual history questions'''
            sexual_history_question_query = """
                                SELECT shq.sexual_history_id, shq.question, shq.response_type
                                FROM sexual_history_question shq
                                INNER JOIN sexual_history_table sh ON shq.sexual_history_id = sh.sexual_history_id;
                            """
            df_sexual_history_question = pd.read_sql_query(sexual_history_question_query, conn)   
            df_sexual_history_question.drop_duplicates(subset=["sexual_history_id","question"],inplace = True)
            df_sexual_history_question = df_sexual_history_question.pivot(index='sexual_history_id', columns='question')
            df_sexual_history_question.columns = df_sexual_history_question.columns.droplevel()
            df_sexual_history_question.reset_index(inplace=True)
            df_sexual_history_question = pd.merge(df_sexual_history_question, df_sexual_history , how="left", on=["sexual_history_id"])
            df_sexual_history_question.rename(columns = {'Exchanged sex for  money/material goods':'Exchanged sex for moneymaterial goods',
                                                            'Victim/ Suspected victim of sexual abuse':'Victim Suspected victim of sexual abuse',
                                                            'Unprotected sex without a condom':'Unprotected sex without a condom',
                                                            'sexually_active': 'Sexually Active' }, inplace = True)
            column_order = {'Event_date': 1, 
                                'person_id':2,'Been incarcerated into jail':3,
                        'Exchanged sex for moneymaterial goods':4,
                        'Had Anal Sex':5,'Had sex with male':6,'Had sex with female':7,'Had sex with a sex worker':8,
                        'Victim Suspected victim of sexual abuse':9,'Tattooed with unsterilized instruments':10,
                        'Received medical injections':11, 'Unprotected sex without a condom':12, 'Injected recreational drugs':13,
                        'History of an STI':14, 'Had Oral Sex':15,  'Received blood transfusions':16, 'Sexually Active':17,
                        'patient_id':18
                                }
            df_sexual_history_question = df_sexual_history_question[[col for col in sorted(df_sexual_history_question.columns, key=lambda x: column_order.get(x, float('inf')))]]
            df_sexual_history_question.drop_duplicates(subset=['patient_id'], keep='last',inplace=True)
            
            '''i. merge dataframe df_positives_with_investigations (13.h) with sexual history'''
            df_positives_complete = pd.merge(df_positives_with_investigations,df_sexual_history_question, on =["patient_id","person_id"] , how = "left")
            df_positives_complete.insert(0,"HIV Positive Province Name",province_name)
            df_positives_complete.insert(1,"HIV Positive District Name",district_name)
            df_positives_complete.insert(2,"HIV Positive Facility Name",facility_name)
            df_positives_complete.insert(3,"HIV Positive Facility Id",facility_id)
            df_positives_complete.insert(4,"HIV Positive DB Name",facility)
            
            df_positives_complete.rename(columns = {"HIV_Result":"Final HIV Result" ,
                                            "reason_for_not_initiating_art":"Reason for not initiating on ART"  },inplace = True)
            
            df_positives_complete.replace(replace_dict, inplace=True)
            df_positives_complete.drop_duplicates(subset=['person_id','Date of HIV Test'], keep='first',inplace=True)
            df_positives_complete.to_sql('positives_complete', conn, index=False)
            ''' j. add df_positives_complete to the global dataframe'''
            global_df_positives = pd.concat([global_df_positives,df_positives_complete])

            '''k. extract data from recency '''
            recency_query = """
                                SELECT 
                                    pi.date as 'Event_date',
                                    pi.person_id, pi.date as 'Date Recency Done',
                                    pi.person_investigation_id,
                                    pi.result as 'Recency Result'
                                FROM person_investigation pi
                                WHERE pi.test = 'HIV-1 Rapid Recency'
                            """            
            df_recency = pd.read_sql_query(recency_query, conn)  
            df_recency.insert(0,"Recency Province Name",province_name)
            df_recency.insert(1,"Recency District Name",district_name)
            df_recency.insert(2,"Recency Facility Name",facility_name)
            df_recency.insert(3,"Recency Facility ID",facility_id)
            df_recency.insert(4,"Recency DB Name",facility)
            df_recency = df_recency.sort_values(by=['Recency Result'])
            df_recency.drop_duplicates(subset=['Event_date','person_id'], keep='first',inplace = True)
            global_df_recency = pd.concat([global_df_recency,df_recency])

            '''l. extract data for demographics'''
            demographics_query = """
                        SELECT 
                            d.person_id,d.firstname as FirstName,d.lastname as LastName, d.birthdate as Birthdate, d.sex as Sex, 
                            d.self_identified_gender as 'Self Identified Gender',
                            d.marital as 'Marital Status',
                            d.education as 'Education',d.occupation as 'Occupation',
                            d.religion as 'Religion',d.nationality_id as 'Nationality Id',
                            d.nationality as Nationality, d.street as 'Street',d.city as City, 
                            d.town as 'Residential Town',i.type as 'Identifier Type' , i.number as 'Identifier Number',p.number as 'Phone Number'
                        FROM demographics d
                        LEFT JOIN identification as i
                            ON d.person_id = i.person_id 
                        LEFT JOIN phone p
                            ON d.person_id = p.person_id
                    """
            df_demographics_extract = pd.read_sql_query(demographics_query, conn) 
            df_demographics_extract.sort_values(['person_id','Identifier Number'],inplace=True)
            df_demographics_extract.drop_duplicates(subset=['person_id'],keep='first',inplace=True)

            ''' m. extract data from patient client profile'''
            if patient_client_profile_df is not None:
                patient_client_profile_query = """
                        SELECT 
                            person_id, client_profile as 'Client Profile', date
                        FROM patient_client_profile
                """
                df_patient_client_profile = pd.read_sql_query(patient_client_profile_query, conn) 
                df_patient_client_profile.sort_values(by=['person_id','date'], inplace=True)
                df_patient_client_profile.drop_duplicates(subset=['person_id'], keep='last',inplace = True)
                df_demographics = pd.merge(df_demographics_extract,df_patient_client_profile,on =["person_id"],how="left")
            else:
                df_demographics = df_demographics_extract
        
            df_demographics.insert(0,"Province Name",province_name)
            df_demographics.insert(1,"District Name",district_name)
            df_demographics.insert(2,"Facility Name",facility_name)
            df_demographics.insert(3,"Facility Id",facility_id)
            df_demographics.insert(4,"Facility DB Name",facility)
            ''' n. add demographics to the global dataframe'''
            global_df_demographics = pd.concat([global_df_demographics,df_demographics])

            ''' o. extract hiv negative data from hts'''
            hts_negative_query =  """
                    SELECT 
                        'hts' as 'Source of Last Negative',
                        h.time as Event_date,
                        p.person_id,
                        h.time as 'Last HIV negative date'
                    FROM hts h  
                    LEFT JOIN person_investigation p
                        ON h.laboratory_investigation_id = p.person_investigation_id
                    WHERE  p.test = 'HIV' AND p.result in ('Negative' ,'NEGATIVE' ,'NEG','Neg')
                """               
            df_hts_negative = pd.read_sql_query(hts_negative_query, conn) 
            df_hts_negative.to_sql('df_hts_negative', conn, index=False)

            '''p. extract hiv negative data from hts screening'''
            hts_screening_negative_query = """
                    SELECT 
                        'HTS Screening' as 'Source of Last Negative',
                        h.date_last_tested as Event_date,
                        p.person_id ,
                        h.date_last_tested as 'Last HIV negative date'
                    FROM hts_screening h
                    LEFT JOIN patient p
                        ON h.patient_id = p.patient_id
                    WHERE result IN ('Negative' ,'NEGATIVE' ,'NEG','Neg')
                """   
            df_hts_screening_negative = pd.read_sql_query(hts_screening_negative_query, conn) 
            df_hts_screening_negative.to_sql('df_hts_screening_negative', conn, index=False) 
               
            '''q. extract hiv negative data from investigation'''
            investigation_negative_query = """
                    SELECT 
                                'Investigation' as 'Source of Last Negative',
                                p.date as Event_date,
                                p.person_id,
                                p.date as 'Last HIV negative date'
                    FROM person_investigation p
                    LEFT JOIN df_hts_negative
                        ON p.person_id = df_hts_negative.person_id
                    LEFT JOIN df_hts_screening_negative
                        ON p.person_id = df_hts_screening_negative.person_id
                    WHERE  test = 'HIV' and result in ('Negative' ,'NEGATIVE' ,'NEG','Neg')  
                        AND df_hts_negative.person_id IS NULL
                        AND df_hts_screening_negative.person_id IS NULL                                
                            
                """
            df_investigation_negative = pd.read_sql_query(investigation_negative_query, conn) 

            '''r. create one dataframe on negatives '''
            df_negatives = pd.concat([df_hts_negative,df_investigation_negative,df_hts_screening_negative])
            df_negatives.insert(0,"Last Negative Province Name",province_name)
            df_negatives.insert(1,"Last Negative District Name",district_name)
            df_negatives.insert(2,"Last Negative Facility Name",facility_name)
            df_negatives.insert(3,"Last Negative Facility ID",facility_id)
            df_negatives.insert(4,"last negative DB Name",facility)
            ''' s. add negatives to the global dataframe'''
            global_df_hts_negative = pd.concat([global_df_hts_negative,df_negatives])

            '''t. extract data from cbs '''
            cbs_query = """
                    SELECT date as 'Event_date' ,
                        person_id,'Yes' as 'Notified', date as 'Date of Notification' , been_on_prep as 'Been On Prep' 
                    FROM cbs 
            """
            df_cbs = pd.read_sql_query(cbs_query, conn) 
            df_cbs.insert(0,"Facility Id",facility_id)
            df_cbs.drop_duplicates(subset=['Event_date','person_id'], keep='last',inplace = True)
            global_df_cbs = pd.concat([global_df_cbs,df_cbs])


            '''u. extract data from art '''
            art_query = """
                    SELECT  date as 'Event_date',
                        art_id,person_id,art_number as "Art Number",art_cohort_number as "Art Cohort Number",
                        date as "Art_Initiation_Date",date_enrolled as "Date Enrolled into ART"
                    FROM art 
                """

            df_art = pd.read_sql_query(art_query, conn) 
            df_art.drop_duplicates(subset=['Event_date','person_id'], keep='last',inplace = True)
            df_art.insert(0,"Art Province Name",province_name)
            df_art.insert(1,"Art District Name",district_name)
            df_art.insert(2,"Art Facility Name",facility_name)
            df_art.insert(3,"Art Facility ID",facility_id)
            df_art.insert(4,"Art DB Name",facility)
            global_df_art = pd.concat([global_df_art,df_art])







            '''u. extract data from art current status '''
            art_query = """
                    SELECT  date as 'Event_date',
                        art_id,person_id,art_number as "Art Number",art_cohort_number as "Art Cohort Number",
                        date as "Art_Initiation_Date2",date_enrolled as "Date Enrolled into ART"
                    FROM art 
                    order by art_id, date
                """
            df_art_current_status2 = pd.read_sql_query(art_current_status_query, conn)
            df_art_current_status2.drop_duplicates(subset=['art_id'], keep='first',inplace = True)
            df_art_current_status2.insert(0,"Art Current Status Province Name",province_name)
            df_art_current_status2.insert(1,"Art Current Status District Name",district_name)
            df_art_current_status2.insert(2,"Art Current Status Facility Name",facility_name)
            df_art_current_status2.insert(3,"Art Current Status Facility ID",facility_id)
            df_art_current_status2.insert(4,"Art Current Status Stage DB Name",facility)
            global_df_art_current_status2 = pd.concat([global_df_art_current_status2,df_art_current_status2])


            

            '''v. extract data from art_visit '''
            art_visit_query = """
                    SELECT p.time as 'Event_date', 
                        p.time as "Art visit date", p.person_id, a.visit_type as 'Art visit type',
                        a.tb_status as 'TB Status', a.family_planning_status, a.lactating_status
                    FROM art_visit a left join patient p
                        ON a.patient_id = p.patient_id
                    
                """  

            df_art_visit = pd.read_sql_query(art_visit_query, conn)
            df_art_visit.drop_duplicates(subset=['Event_date','person_id'], keep='last',inplace = True)
            df_art_visit = df_art_visit.replace('Presumptive-if there are signs', 'Suspect or Presumptive')
            df_art_visit = df_art_visit.replace('Screened and has no signs', 'Screened with no signs')
            df_art_visit = df_art_visit.replace('Tb status not assesssed', '')
            df_art_visit.insert(0,"Art Visit Province Name",province_name)
            df_art_visit.insert(1,"Art Visit District Name",district_name)
            df_art_visit.insert(2,"Art Visit Facility Name",facility_name)
            df_art_visit.insert(3,"Art Visit Facility ID",facility_id)
            df_art_visit.insert(4,"Art Visit DB Name",facility)
            global_df_art_visit = pd.concat([global_df_art_visit,df_art_visit])


            '''w. extract data from art_current_status '''
            art_current_status_query = """
                    SELECT a.date as 'Event_date', 
                        a.date as 'Art Current Status Date',a.art_id ,a.regimen as regimen , a.state as 'ARV Status', a.art_initiation_category as 'Art Initiation Category',
                        art.person_id,a.regimen_id as 'Regimen Id'
                    FROM art_current_status a 
                    LEFT JOIN art 
                        ON a.art_id = art.art_id 
                """     

            df_art_current_status = pd.read_sql_query(art_current_status_query, conn)
            df_art_current_status.drop_duplicates(subset=['Event_date','person_id'], keep='last',inplace = True)
            df_art_current_status.insert(0,"Art Current Status Province Name",province_name)
            df_art_current_status.insert(1,"Art Current Status District Name",district_name)
            df_art_current_status.insert(2,"Art Current Status Facility Name",facility_name)
            df_art_current_status.insert(3,"Art Current Status Facility ID",facility_id)
            df_art_current_status.insert(4,"Art Current Status Stage DB Name",facility)
            global_df_art_current_status = pd.concat([global_df_art_current_status,df_art_current_status])

            '''x. extract data from art who stage '''
            art_who_stage_query = """
                        SELECT  a.date as 'Event_date',
                            p.person_id , a.art_id, a.date as 'Who Stage Date',
                            a.date as 'Outcome Date',
                            a.stage as 'Who Stage', a.follow_up_status as 'Art Outcome'
                        FROM art_who_stage a left join art p
                            on a.art_id  = p.art_id
                        
                    """

            df_art_who_stage = pd.read_sql_query(art_who_stage_query, conn)
            df_art_who_stage.drop_duplicates(subset=['Event_date','person_id'], keep='last',inplace = True)
            df_art_who_stage.insert(0,"Who stage Province Name",province_name)
            df_art_who_stage.insert(1,"Who stage District Name",district_name)
            df_art_who_stage.insert(2,"Who stage Facility Name",facility_name)
            df_art_who_stage.insert(3,"Who stage Facility ID",facility_id)
            df_art_who_stage.insert(4,"Who Stage DB Name",facility)
            global_df_art_who_stage = pd.concat([global_df_art_who_stage,df_art_who_stage])

            '''y. extract data from viral load '''
            viral_load_query = """
                    SELECT date as "Event_date",
                        person_id,date as "Date at which Viral Load result was issued",
                        date as "Date for which Viral Load was taken",
                        'TRUE' as "Viral Load Sample submitted to lab",
                        'TRUE' as "Was Viral Load result issued",
                        result as "Viral Load result"
                    FROM person_investigation 
                    WHERE test = 'Viral Load'  
                    
                """
            df_viral_load = pd.read_sql_query(viral_load_query, conn) 
            df_viral_load.drop_duplicates(subset=['Event_date','person_id'], keep='last', inplace = True)
            df_viral_load.insert(0,"Viral Load Province Name",province_name)
            df_viral_load.insert(1,"Viral Load District Name",district_name)
            df_viral_load.insert(2,"Viral Load Facility Name",facility_name)
            df_viral_load.insert(3,"Viral Load Facility ID",facility_id)
            df_viral_load.insert(4,"Viral Load DB Name",facility)
            global_df_viral_load = pd.concat([global_df_viral_load,df_viral_load])

            '''z. extract data from cd4 '''
            cd4_query = """
                    SELECT date as "Event_date",
                        person_id, date as 'Date at which cd4 sample was taken',
                        date as 'Date at which cd4 result was issued',
                        'TRUE' as "CD4 Sample submitted to lab",
                        'TRUE' as "Was cd4 result issued",
                        result as 'CD4 Count'
                    FROM person_investigation where test = 'CD4 Count' 
                """       

            df_cd4 = pd.read_sql_query(cd4_query, conn)

            df_cd4.drop_duplicates(subset=['Event_date','person_id'], keep='last',inplace = True)
            df_cd4.insert(0,"cd4 Province Name",province_name)
            df_cd4.insert(1,"cd4 District Name",district_name)
            df_cd4.insert(2,"cd4 Facility Name",facility_name)
            df_cd4.insert(3,"cd4 Facility ID",facility_id)
            df_cd4.insert(4,"cd4 DB Name",facility)
            global_df_cd4 = pd.concat([global_df_cd4,df_cd4])

            '''aa. extract data from cd4 '''
            TB_query = """
                    SELECT date as 'Event_date',
                        person_id,date as 'TB Treatment Start Date',type_of_tb as 'Type Of TB',
                        tb_disease_site as 'TB Disease Site',
                        tb_disease_type as 'TB Disease Type',outcome as 'TB Outcome'
                    FROM tb 
                """

            df_TB = pd.read_sql_query(TB_query, conn)
            df_TB.drop_duplicates(subset=['Event_date','person_id'], keep='last',inplace =  True)
            df_TB.insert(0,"TB Province Name",province_name)
            df_TB.insert(1,"TB District Name",district_name)
            df_TB.insert(2,"TB Facility Name",facility_name)
            df_TB.insert(3,"TB Facility Id",facility_id)
            df_TB.insert(4,"TB DB Name",facility)
            global_df_tb = pd.concat([global_df_tb,df_TB])

            '''ab. extract data from TB screening '''
            TB_Screening_query = """
                    SELECT t.time as 'Event_date',
                        t.person_id , p.presumptive as 'TB Screened'
                    FROM tb_screening p
                    LEFT JOIN patient t 
                        on p.patient_id = t.patient_id
                """  

            df_TB_Screening = pd.read_sql_query(TB_Screening_query, conn)      
        
            df_TB_Screening = df_TB_Screening.replace(1, 'Suspect or Presumptive')
            df_TB_Screening = df_TB_Screening.replace(0, 'Screened with no signs')
            
            df_TB_Screening.drop_duplicates(subset=['Event_date','person_id'], keep='last',inplace =  True)
            df_TB_Screening.insert(0,"TB Screening Province Name",province_name)
            df_TB_Screening.insert(1,"TB Screening District Name",district_name)
            df_TB_Screening.insert(2,"TB Screening Facility Name",facility_name)
            df_TB_Screening.insert(3,"TB Screening Facility Id",facility_id)
            df_TB_Screening.insert(4,"TB Screening DB Name",facility)
            global_df_tb_screening = pd.concat([global_df_tb_screening,df_TB_Screening])

            '''ac. extract data from transfer out '''
            transfer_out_query = """
                    SELECT 
                        a.person_id , t.art_id , t.transfer_out_date as 'Event_date',
                        t.transfer_out_date as 'Transfer Out date',
                        t.transfer_reason as 'Transfer Reason', t.transfer_facility_id
                    FROM art_transfer_out t 
                    LEFT JOIN art a
                        ON t.art_id = a.art_id
                """

            df_transfer_out = pd.read_sql_query(transfer_out_query, conn)
            df_transfer_out.drop_duplicates(subset=['Event_date','person_id'], keep='last',inplace = True)
            df_transfer_out.insert(0,"Transfer Out Province Name",province_name)
            df_transfer_out.insert(1,"Transfer Out District Name",district_name)
            df_transfer_out.insert(2,"Transfer Out Facility Name",facility_name)
            df_transfer_out.insert(3,"Transfer Out Facility Id",facility_id)
            global_df_transfer_out = pd.concat([global_df_transfer_out,df_transfer_out])
        
            '''ad. Extract data to use for merging with NMRL viral load data '''
            laboratory_request_order_query = """
                                            SELECT p.person_id, 
                                                d.firstname,d.lastname,d.birthdate,d.sex,
                                                l.laboratory_request_order_id,
                                                l.laboratory_request_number,
                                                l.laboratory_investigation_id,
                                                l.facility_id,
                                                l.request_reason,
                                                l.laboratory,
                                                l.date_sample_taken,
                                                l.result,
                                                l.result_date,
                                                l.result_issued,
                                                a.art_id,
                                                a.art_number
                                            FROM laboratory_request_order l
                                            LEFT JOIN person_investigation p
                                                on l.laboratory_investigation_id = p.person_investigation_id
                                            LEFT JOIN art a
                                                on p.person_id = a.person_id
                                            LEFT JOIN demographics d
                                                on p.person_id = d.person_id
                                                """
            df_laboratory_request_order = pd.read_sql_query(laboratory_request_order_query, conn)
            global_df_laboratory_request = pd.concat([global_df_laboratory_request,df_laboratory_request_order])

            '''ae. clean lab request number'''
            print("   >>> clean lab request number")
            df_laboratory_request_order.loc[:,'clean_laboratory_request_number'] = df_laboratory_request_order["laboratory_request_number"].astype(str).str.replace('\W', '', regex=True).apply(lambda x: x.lower().replace(' ',''))
            df_nmrl.loc[:, 'clean_laboratory_request_number'] = df_nmrl["client sample ID"].astype(str).str.replace('\W', '', regex=True).apply(lambda x: x.lower().replace(' ',''))
            
            '''af. merging nmrl data on lab request number'''
            print("   >>> merging on lab request number")
            df_nmrl_on_lab_id = pd.DataFrame()
            df_nmrl_on_lab_id_facility =  df_nmrl[df_nmrl['Facility Id'] == facility_id]
            if df_nmrl_on_lab_id_facility.shape[0] != 0:
                df_nmrl_on_lab_id , df_nmrl = cbs_extraction.match_on_lab(
                    df_laboratory_request_order,
                    df_nmrl_on_lab_id_facility, df_nmrl
                )
                df_nmrl_on_lab_id.insert(0,"Viral Load Province Name",province_name)
                df_nmrl_on_lab_id.insert(1,"Viral Load District Name",district_name)
                df_nmrl_on_lab_id.insert(2,"Viral Load Facility Name",facility_name)
                df_nmrl_on_lab_id.insert(3,"Viral Load Facility ID",facility_id)
                df_nmrl_on_lab_id.insert(4,"Viral Load DB Name",facility)
                global_df_vl_nmrl_lab = pd.concat([global_df_vl_nmrl_lab,df_nmrl_on_lab_id])
                print("nmrl new size ", df_nmrl.shape)

            '''ag. clean art numbers'''
            # Clean art numbers...............................................................................
            print("    >>> cleaning art number")
            df_art = df_art.fillna('empty string')
            df_art = df_art.replace('', 'empty string')
            df_art.loc[:,'clean_art_number'] = df_art["Art Number"].astype(str).str.replace('\W', '', regex=True).apply(lambda x: x.lower().replace(' ',''))

            '''ah. merging nmrl data on art number'''
            exact_matches = {}
            print("   Total positive people >>>", df_positives.shape[0])
            print("   >>> matching on art")
            df_match_art, df_nmrl = cbs_extraction.match_on_art_number( df_art , df_nmrl)
            global_df_vl_nmrl_art = pd.concat([global_df_vl_nmrl_art,df_match_art])

            print("    >>> NMRL new size ", df_nmrl.shape)
            
            '''ah. merging nmrl data on demographics'''
            print("    >>> exact matches between NMRL Demographics and EHR Demographics")
            ''' Find exact matches between NMRL Facility/Lab/Demographics and EHR Laboratory Investigation ID '''
            
            # df_match_demos['Birthdate'] = cbs_extraction.clean_dates(df_nmrl_on_lab_id, 'DOB')
            df_demographics['Birthdate_cleaned'] = cbs_extraction.clean_dates(df_demographics, 'Birthdate')
            df_nmrl['Birthdate_cleaned'] = df_nmrl['Birthdate']
            df_match_demos, df_nmrl  = cbs_extraction.match_on_demos(
                                    df_demographics, 
                                    df_nmrl)
            
            print("    >>> NMRL new size ", df_nmrl.shape)
            
            global_df_vl_nmrl_demos = pd.concat([global_df_vl_nmrl_demos ,df_match_demos])  

            # Mother baby pair.......................................................................................
            print("   >>> Extracting babies who tested positive")
            positive_babies_mother_sql = """
                                    WITH ranked_messages AS (
                                        SELECT ps.person_id as 'Baby Person id',
                                            ps.firstname as 'Baby FirstName',
                                            ps.lastname as 'Baby LastName',
                                            ps.birthdate as 'Baby Birthdate',
                                            ps.sex as 'Baby Sex',
                                            pi.date as 'Baby Date of HIV Test', 
                                            pi.result as 'HIV Result of Baby',
                                            pi.test as 'HIV test for Baby',
                                            pm.person_id as 'Mother Person id' ,
                                            pm.firstname as 'Mother FirstName',
                                            pm.lastname as 'Mother LastName',
                                            pm.birthdate as 'Mother Birthdate',
                                            pm.sex as 'Mother Sex',
                                            ROW_NUMBER() OVER (PARTITION BY ps.person_id ORDER BY pi.date ASC) as ranking
                                            FROM client.person ps
                                            left join consultation.person_investigation pi                                        
                                            on ps.person_id = pi.person_id
                                            left join consultation.pregnancy_live_birth pb                                       
                                            on pi.person_id = pb.person_id
                                            left join consultation.pregnancy pg                                      
                                            on pb.pregnancy_id = pg.pregnancy_id
                                            left join client.person pm                                   
                                            on pg.person_id = pm.person_id
                                            where pi.result = 'positive'
                                            and (pi.test = 'hiv' or pi.test = 'HIV DNA PCR')
                                            AND DATEDIFF(pi.date, ps.birthdate) <= 730
                                                )
                                                SELECT * FROM ranked_messages where ranking = 1
                                    """
            positive_babies_mother = cbs_extraction.extracting_data(positive_babies_mother_sql, connection)
            status_of_mother = cbs_extraction.extracting_data("""
                                            select person_id as 'Mother Person id', date as 'Mother Date of HIV Test ',
                                            result as 'HIV Result of Mother' from consultation.person_investigation
                                            where test in ('HIV','HIV DNA PCR')
                                            """,connection)
            status_of_mother_art = cbs_extraction.extracting_data("""
                                            select person_id as 'Mother Person id', date_of_hiv_test as 'Mother Date of HIV Test (ART)',
                                            'Positive' as 'HIV Result of Mother (ART)' from consultation.art
                                            """,connection)
            merged_df = positive_babies_mother.merge(status_of_mother, on='Mother Person id', how='left')
            merged_df = merged_df.merge(status_of_mother_art, on='Mother Person id', how='left')
            merged_df["Facility id"] = facility_id
            merged_df["Facility Name"] = facility
            merged_df.drop('ranking', axis=1, inplace=True)
            global_baby_to_mother = pd.concat([global_baby_to_mother,merged_df])

            # a. Method 1: Link babies who are less than or equal to 24 months old and HIV positive with their mothers.
            # and Method 1 b: Link the remaining babies to their mothers based on their relationship.
            positive_babies_mother_relation_sql = """
                                    WITH ranked_messages AS (
                                        SELECT ps.person_id as 'Baby Person id',
                                            ps.firstname as 'Baby FirstName',
                                            ps.lastname as 'Baby LastName',
                                            ps.birthdate as 'Baby Birthdate',
                                            ps.sex as 'Baby Sex',
                                            pi.date as 'Baby Date of HIV Test', 
                                            pi.result as 'HIV Result of Baby',
                                            pi.test as 'HIV test for Baby',
                                            pm.person_id as 'Mother Person id' ,
                                            pm.firstname as 'Mother FirstName',
                                            pm.lastname as 'Mother LastName',
                                            pm.birthdate as 'Mother Birthdate',
                                            pm.sex as 'Mother Sex',
                                            ROW_NUMBER() OVER (PARTITION BY ps.person_id ORDER BY pi.date ASC) as ranking
                                            FROM client.person ps
                                            left join consultation.person_investigation pi                                        
                                            on ps.person_id = pi.person_id
                                            left join client.relationship rs                                      
                                            on pi.person_id = rs.member_id
                                            left join client.person pm                                   
                                            on rs.person_id = pm.person_id
                                            where pi.result = 'positive' and rs.relation = 'CHILD'
                                            and (pi.test = 'hiv' or pi.test = 'HIV DNA PCR')
                                            AND DATEDIFF(pi.date, ps.birthdate) <= 730
                                                )
                                                SELECT * FROM ranked_messages where ranking = 1
                                    """
            positive_babies_mother_relation = cbs_extraction.extracting_data(positive_babies_mother_relation_sql, connection)
            positive_babies_mother_relation["Facility id"] = facility_id
            positive_babies_mother_relation["Facility Name"] = facility
            positive_babies_mother_relation.drop('ranking', axis=1, inplace=True)
            # global_babies_mother_relation = pd.concat([global_babies_mother_relation,positive_babies_mother_relation])
            
            #Method 2:
            # 1. Extract demographics of all females who test positive for HIV and (pregnant or breast feeding).
            # 2a. Link the mothers to their babies if the mothers delivered at the same facility where they tested positive.
            positive_mothers_preg_or_lactating_sql = """
                                    WITH ranked_messages AS (
                                        SELECT ps.person_id as 'Mother Person id',
                                            ps.firstname as 'Mother FirstName',
                                            ps.lastname as 'Mother LastName',
                                            ps.birthdate as 'Mother Birthdate',
                                            ps.sex as 'Mother Sex',
                                            h.time as 'Mother Date of HIV Test', 
                                            pi.result as 'HIV Result of Mother',
                                            ps2.firstname as 'Baby FirstName',
                                            ps2.lastname as 'Baby LastName',
                                            ps2.birthdate as 'Baby Birthdate',
                                            ps2.sex as 'Baby Sex', 
                                            'Alive' as 'Delivery Status',
                                            plb.delivery_mode as 'Delivery mode',
                                            plb.weight as 'Weight of Baby',
                                            ROW_NUMBER() OVER (PARTITION BY ps.person_id ORDER BY pi.date ASC) as ranking
                                            FROM consultation.hts h
                                            left join consultation.person_investigation pi
                                            on h.laboratory_investigation_id = pi.person_investigation_id
                                            left join client.person ps
                                            on pi.person_id = ps.person_id
                                            left join consultation.pregnancy pgg
                                            on pi.person_id = pgg.person_id
                                            left join consultation.pregnancy_live_birth plb
                                            on pgg.pregnancy_id = plb.pregnancy_id
                                            left join client.person ps2
                                            on plb.person_id = ps2.person_id
                                            where pi.result = 'positive'
                                            and (pi.test = 'hiv' or pi.test = 'HIV DNA PCR')
                                            AND (pregnant = 1 or lactating = 1)
                                            )
                                                SELECT * FROM ranked_messages where ranking = 1
                                    """
            positive_mothers_live_birth = cbs_extraction.extracting_data(positive_mothers_preg_or_lactating_sql, connection)
            positive_mothers_live_birth.drop('ranking', axis=1, inplace=True)
            positive_mothers_live_birth['Delivery Status'] = positive_mothers_live_birth.apply(lambda row: '' if pd.isnull(row['Baby Sex']) else row['Delivery Status'], axis=1)

            # Still Birth
            positive_mothers_preg_or_lactating_sql = """
                                    WITH ranked_messages AS (
                                        SELECT ps.person_id as 'Mother Person id',
                                            ps.firstname as 'Mother FirstName',
                                            ps.lastname as 'Mother LastName',
                                            ps.birthdate as 'Mother Birthdate',
                                            ps.sex as 'Mother Sex',
                                            h.time as 'Mother Date of HIV Test', 
                                            pi.result as 'HIV Result of Mother',
                                            plb.time as 'Still Birth date',
                                            plb.sex as 'Baby Sex', 
                                            'Still Birth' as 'Delivery Status',
                                            plb.delivery_mode as 'Delivery mode',
                                            plb.weight as 'Weight of Baby',
                                            ROW_NUMBER() OVER (PARTITION BY ps.person_id ORDER BY pi.date ASC) as ranking
                                            FROM consultation.hts h
                                            left join consultation.person_investigation pi
                                            on h.laboratory_investigation_id = pi.person_investigation_id
                                            left join client.person ps
                                            on pi.person_id = ps.person_id
                                            left join consultation.pregnancy pgg
                                            on pi.person_id = pgg.person_id
                                            left join consultation.pregnancy_still_birth plb
                                            on pgg.pregnancy_id = plb.pregnancy_id
                                            where pi.result = 'positive'
                                            and (pi.test = 'hiv' or pi.test = 'HIV DNA PCR')
                                            AND (pregnant = 1 or lactating = 1)
                                            )
                                            SELECT * FROM ranked_messages where ranking = 1
                                    """
            positive_mothers_still_birth = cbs_extraction.extracting_data(positive_mothers_preg_or_lactating_sql, connection)
            positive_mothers_still_birth['Delivery Status'] = positive_mothers_still_birth.apply(lambda row: '' if pd.isnull(row['Baby Sex']) else row['Delivery Status'], axis=1)

            df_positive_mother =  pd.concat([positive_mothers_live_birth,positive_mothers_still_birth])
            df_positive_mother["Facility id"] = facility_id
            df_positive_mother["Facility Name"] = facility
            
            df_positive_mother.drop_duplicates(keep='first',inplace=True)
            df_positive_mother.drop('ranking', axis=1, inplace=True)
            # global_positive_mother = pd.concat([global_positive_mother,df_positive_mother])

            #Method 2:
            # 2b. Link the mothers to their children through their relationship.
            #    - Link the "person" table with the "relationship" table based on the "person_id" field.
            positive_mothers_relationship_sql = """
                                    WITH ranked_messages AS (
                                        SELECT ps.person_id as 'Mother Person id',
                                            ps.firstname as 'Mother FirstName',
                                            ps.lastname as 'Mother LastName',
                                            ps.birthdate as 'Mother Birthdate',
                                            ps.sex as 'Mother Sex',
                                            h.time as 'Mother Date of HIV Test', 
                                            pi.result as 'HIV Result of Mother',
                                            ps2.person_id as 'Baby Person id',
                                            ps2.firstname as 'Baby FirstName',
                                            ps2.lastname as 'Baby LastName',
                                            ps2.birthdate as 'Baby Birthdate',
                                            ps2.sex as 'Baby Sex',
                                            'MOTHER' as relation,
                                            ROW_NUMBER() OVER (PARTITION BY ps.person_id ORDER BY pi.date ASC) as ranking
                                            FROM consultation.hts h
                                            left join consultation.person_investigation pi
                                            on h.laboratory_investigation_id = pi.person_investigation_id
                                            left join client.person ps
                                            on pi.person_id = ps.person_id
                                            left join client.relationship rs
                                            on pi.person_id = rs.person_id
                                            left join client.person ps2
                                            on rs.member_id = ps2.person_id
                                            where pi.result = 'positive'
                                            and (pi.test = 'hiv' or pi.test = 'HIV DNA PCR')
                                            AND (pregnant = 1 or lactating = 1)
                                            and rs.relation = 'CHILD'
                                            )
                                                SELECT * FROM ranked_messages where ranking = 1
                                    """
            positive_mothers_relation = cbs_extraction.extracting_data(positive_mothers_relationship_sql, connection)
            
            positive_mothers_relation.drop('ranking', axis=1, inplace=True)

            babies_relationship_sql = """
                                    WITH ranked_messages AS (
                                        SELECT ps.person_id as 'Baby Person id',
                                            pi.date as 'Baby Date of HIV Test',
                                            pi.result as 'HIV Result of Baby',
                                            ROW_NUMBER() OVER (PARTITION BY ps.person_id ORDER BY pi.date ASC) as ranking
                                            FROM consultation.person_investigation pi
                                            left join client.person ps
                                            on pi.person_id = ps.person_id
                                            where test in ('hiv','HIV DNA PCR')
                                            )
                                                SELECT * FROM ranked_messages where ranking = 1
                                    """
            baby_relation = cbs_extraction.extracting_data(babies_relationship_sql, connection)

            positive_mothers_relation = pd.merge(positive_mothers_relation,baby_relation,on = ['Baby Person id'], how = 'left')
            positive_mothers_relation["Facility id"] = facility_id
            positive_mothers_relation["Facility Name"] = facility

            # global_mothers_relation = pd.concat([global_mothers_relation,positive_mothers_relation])

            mother_to_baby = pd.concat([df_positive_mother,positive_mothers_relation])

            global_mother_to_baby = pd.concat([global_mother_to_baby,mother_to_baby])

            if not df_match_art.empty:
                df_match_art_len = len(set(df_match_art['person_id']))
            else:
                df_match_art_len = 0
            
            if not df_match_demos.empty:
                df_match_demos_len = len(set(df_match_demos['person_id']))
            else:
                df_match_demos_len = 0
            
            if not df_nmrl_on_lab_id.empty:
                df_nmrl_on_lab_id_len = len(set(df_nmrl_on_lab_id['person_id']))
            else:
                df_nmrl_on_lab_id_len = 0


            df_stats = pd.DataFrame({"Province": [province_name],
                                    "District": [district_name],
                                    "Facility": [facility_name],
                                    "Facility id": [facility_id],
                                    "Number of positive people": df_positives.shape[0],
                                    "Number of matches found using ART": df_match_art_len,
                                    "Number of matches found using demos": df_match_demos_len,
                                    "Number of matches found using lab number": df_nmrl_on_lab_id_len
                                    })
            global_df_stats = pd.concat([global_df_stats, df_stats])
            
            # save reduced nmrl file
            df_nmrl.to_feather("nmrl_reduced.feather")
            df_nmrl.to_csv("nmrl_reduced.csv")
            # Save globals in case there is an error in extraction 
            # saved to csv for any validation to be done 
            print("  >>> Saving global dataframes to csv files")
            # if os.path.exists("global_df_vl_nmrl_art.csv"):
            #     df_stats.to_csv("Stats.csv", mode = 'a', header = False, index= False)
            #      global_df_vl_nmrl_lab.to_csv("global_vl_nmrl_lab.csv", mode = 'a', header = False, index = False)
            #     global_df_vl_nmrl_art.to_csv("global_df_vl_nmrl_art.csv", mode = 'a', header = False, index = False)
            #     global_df_vl_nmrl_demos.to_csv("global_df_vl_nmrl_demos.csv", mode = 'a', header = False, index = False)
            #     global_df_demographics.to_csv("Demographics.csv",mode='a', header = False, index = False)
            #     global_df_positives.to_csv("global_df_positives.csv",mode='a', header = False, index = False)
            #     global_df_hts_negative.to_csv("global_df_negative.csv",mode='a', header = False, index = False)
            #     global_df_cbs.to_csv("global_df_cbs.csv",mode='a', header = False, index = False)
            #     global_df_recency.to_csv("global_df_recency.csv",mode='a', header = False, index = False)
            #     global_df_art.to_csv("global_df_art.csv",mode='a', header = False, index = False)
            #     global_df_art_visit.to_csv("global_df_art_visit.csv",mode='a', header = False, index = False)
            #     global_df_art_current_status.to_csv("global_df_art_current_status.csv",mode='a', header = False, index = False)
            #     global_df_viral_load.to_csv("global_df_viral_load.csv",mode='a', header = False, index = False)
            #     global_df_cd4.to_csv("global_df_cd4.csv",mode='a',  header = False, index = False)
            #     global_df_tb.to_csv("global_df_tb.csv",mode='a',  header = False, index = False)
            #     global_df_tb_screening.to_csv("global_df_tb_screening.csv",mode='a',  header = False, index = False)
            #     global_df_transfer_out.to_csv("global_df_transfer_out.csv",mode='a', header = False, index = False)
            #     global_df_art_who_stage.to_csv("global_df_art_who_stage.csv",mode='a', header = False, index = False)
            #     global_df_laboratory_request.to_csv("global_df_laboratory_request.csv",mode='a',  header = False, index = False)
            #     global_mother_to_baby.to_csv("global_mother_to_baby.csv",mode='a',  header = False, index = False)
            #     global_baby_to_mother.to_csv("global_baby_to_mother.csv",mode='a',  header = False, index = False)
            
            # else:
            #     # Write data to the CSV file with headers
            global_df_stats.to_csv("Stats.csv", index= False)
            global_df_vl_nmrl_lab.to_csv("global_vl_nmrl_lab.csv", index = False)
            global_df_vl_nmrl_art.to_csv("global_df_vl_nmrl_art.csv", index = False)
            global_df_vl_nmrl_demos.to_csv("global_df_vl_nmrl_demos.csv", index = False)
            global_df_demographics.to_csv("Demographics.csv", index = False)
            global_df_positives.to_csv("global_df_positives.csv", index = False)
            global_df_hts_negative.to_csv("global_df_negative.csv", index = False)
            global_df_cbs.to_csv("global_df_cbs.csv",index = False)
            global_df_recency.to_csv("global_df_recency.csv", index = False)
            global_df_art.to_csv("global_df_art.csv", index = False)
            global_df_art_visit.to_csv("global_df_art_visit.csv", index = False)
            global_df_art_current_status.to_csv("global_df_art_current_status.csv", index = False)
            global_df_viral_load.to_csv("global_df_viral_load.csv", index = False)
            global_df_cd4.to_csv("global_df_cd4.csv" , index = False)
            global_df_tb.to_csv("global_df_tb.csv" , index = False)
            global_df_tb_screening.to_csv("global_df_tb_screening.csv", index = False)
            global_df_transfer_out.to_csv("global_df_transfer_out.csv", index = False)
            global_df_art_who_stage.to_csv("global_df_art_who_stage.csv", index = False)
            global_df_laboratory_request.to_csv("global_df_laboratory_request.csv", index = False)
            global_mother_to_baby.to_csv("global_mother_to_baby.csv", index = False)
            global_baby_to_mother.to_csv("global_baby_to_mother.csv", index = False)
            global_df_art_current_status2.to_csv("global_df_art_current_status2.csv", index = False)
    
            # Delete trimmed database
            os.remove(trimmed_db)

            if db_size_left <= 0:
                db_size_left = 0 
            print(f"size left {round(db_size_left,2)} GB")
            cbs_extraction.log(processing_time,province_name,district_name,facility_name,facility_id,facility,"","",db_size,"","Completed","web","","")
         
        except Exception as e:  # noqa: E722
            print (e)
            cbs_extraction.log(processing_time,"","","","",facility,"","",db_size,"","Failed : "+ str(e),"web","","")
            continue

    print("  >>> Saving global dataframes to feather files")
  


    # global_mother_to_baby = pd.read_csv("global_mother_to_baby.csv", index = False)
    # global_mother_to_baby.to_feather("global_mother_to_baby.feather")

    # global_baby_to_mother = pd.read_csv("global_baby_to_mother.csv")
    # global_df_vl_nmrl_lab.to_feather("global_baby_to_mother.feather")

    # global_df_vl_nmrl_lab = pd.read_csv('global_vl_nmrl_lab.csv')
    # global_df_vl_nmrl_lab.to_feather("global_vl_nmrl_lab.feather")

    # global_df_vl_nmrl_art = pd.read_csv('global_df_vl_nmrl_art.csv')
    # global_df_vl_nmrl_art.to_feather("global_df_vl_nmrl_art.feather")

    # global_df_vl_nmrl_demos = pd.read_csv('global_df_vl_nmrl_demos.csv')
    # global_df_vl_nmrl_demos.to_feather("global_df_vl_nmrl_demos.feather")

    # global_df_demographics = pd.read_csv('Demographics.csv')
    # global_df_demographics.to_feather("Demographics.feather")

    # global_df_positives = pd.read_csv('global_df_positives.csv')
    # global_df_positives.to_feather("global_df_positives.feather")

    # global_df_hts_negative = pd.read_csv('global_df_negative.csv')
    # global_df_hts_negative.to_feather("global_df_negative.feather")

    # global_df_cbs = pd.read_csv('global_df_cbs.csv')
    # global_df_cbs.to_feather("global_df_cbs.feather")

    # global_df_recency = pd.read_csv('global_df_recency.csv')
    # global_df_recency.to_feather("global_df_recency.feather")

    # global_df_art = pd.read_csv('global_df_art.csv')
    # global_df_art.to_feather("global_df_art.feather")

    # global_df_art_visit = pd.read_csv('global_df_art_visit.csv')
    # global_df_art_visit.to_feather("global_df_art_visit.feather")

    # global_df_art_current_status = pd.read_csv('global_df_art_current_status.csv')
    # global_df_art_current_status.to_feather("global_df_art_current_status.feather")

    # global_df_viral_load = pd.read_csv('global_df_viral_load.csv')
    # global_df_viral_load.to_feather("global_df_viral_load.feather")

    # global_df_cd4 = pd.read_csv('global_df_cd4.csv')
    # global_df_cd4.to_feather("global_df_cd4.feather")

    # global_df_tb = pd.read_csv('global_df_tb.csv')
    # global_df_tb.to_feather("global_df_tb.feather")

    # global_df_tb_screening = pd.read_csv('global_df_tb_screening.csv')
    # global_df_tb_screening.to_feather("global_df_tb_screening.feather")

    # global_df_transfer_out = pd.read_csv('global_df_transfer_out.csv')
    # global_df_transfer_out.to_feather("global_df_transfer_out.feather")

    # global_df_art_who_stage = pd.read_csv('global_df_art_who_stage.csv')
    # global_df_art_who_stage.to_feather("global_df_art_who_stage.feather")

    # global_df_laboratory_request = pd.read_csv('global_df_laboratory_request.csv')
    # global_df_laboratory_request.to_feather("global_df_laboratory_request.feather")

    # df_ehr = pd.read_feather("Demographics.feather")
    # df_nmrl = pd.read_feather('nmrl_reduced.feather')


  

NMRL file not found
>>> Number of databases  1
>>> Total size: 0.67  GB
.........................................................................................................
>>> 1 / 1 ./dbs/Nyamutumbu09April24.sql
name 'df_nmrl' is not defined


NameError: name 'processing_time' is not defined

Fomartting 